# Generate Masks for Time Series Data

## Setup

In [ ]:
if "NOTEBOOK_INITIATED_FLAG" not in globals():
    NOTEBOOK_INITIATED_FLAG = True
    %cd ..
    !pip3 install git+https://github.com/pvigier/perlin-numpy astropy geopy
%pwd

import h5py
import os
import numpy as np
import matplotlib.pyplot as plt

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

plt.rcParams["figure.figsize"] = (16, 8)

from tensorflow.keras.models import load_model
from osgeo import gdal
from PIL import Image
from insar_eventnet.io import get_image_array
from insar_eventnet.inference import mask_with_model, mask_simulated
from insar_eventnet.sarsim import (
    gen_simulated_deformation,
    gen_sim_noise,
    gen_simulated_time_series,
)

: 

## Load Data

In [ ]:
timeseries_filename = f"notebooks/{input('path to timeseries h5 file: ')}"
unwrapped = h5py.File(timeseries_filename, "r").get("/timeseries")[...]

print(unwrapped[1].shape)

: 

## Show Data

In [ ]:
plt.imshow(unwrapped[1])

: 

In [ ]:
plt.imshow(unwrapped[100])

: 

## Process Data

In [ ]:
tile_size = 512
crop_size = 512

wrapped = np.angle(np.exp(1j * (unwrapped)))

# Currently crops data to fit into model instead of tiling
z, x, y = wrapped.shape
wrapped = wrapped[:16, x // 2 - 256 : x // 2 + 256, y // 2 - 256 : y // 2 + 256]

plt.imshow(wrapped[1])

: 

## Run Prediction

In [ ]:
tile_size = 512
crop_size = 512

mask_model_path = "notebooks/models/time_series_masking_model"

mask_model = load_model(mask_model_path)

wrapped = wrapped.transpose(1, 2, 0)
wrapped = wrapped.reshape((1, *wrapped.shape, 1))
mask_pred = np.float32(mask_model.predict(wrapped))

: 

## Process Mask

In [ ]:
mask_accumulated = np.zeros((512, 512))
for i in range(16):
    mask_accumulated += mask_pred[0, :, :, i, 0]
plt.imshow(np.angle(np.exp(1j * mask_accumulated)))

mask = mask_accumulated
mask[mask == 0] = 0
mask[mask >= 0.5] = 1
mask[mask < 0.5] = 0

: 

## Plot Mask

In [ ]:
plt.imshow(np.angle(np.exp(1j * mask)))

: 